In [1]:
!pip install pip install openai==0.28 langchain faiss-cpu tiktoken requests scrapy selenium

## Downloading IMDB Reviews
1. Barbie reviews
2. Openheimer reviews

- Sometimes when trying to excute them sequentially you will get an error, so run one then restart run time and run the remaining one

In [2]:
import numpy as np
import pandas as pd
from scrapy.selector import Selector
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")


chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=chrome_options)

In [4]:
# Barbie IMDB reviews

url = "https://www.imdb.com/title/tt1517268/reviews/?ref_=tt_ov_rt"
driver.get(url)


sel = Selector(text = driver.page_source)
review_counts = sel.css('.lister .header span::text').extract_first().replace(',','').split(' ')[0]
more_review_pages = int(int(review_counts)/25)

for i in tqdm(range(more_review_pages)):
    try:
        css_selector = 'load-more-trigger'
        driver.find_element(By.ID, css_selector).click()
    except:
        pass

rating_list = []
review_date_list = []
review_title_list = []
author_list = []
review_list = []
review_url_list = []
error_url_list = []
error_msg_list = []
reviews = driver.find_elements(By.CSS_SELECTOR, 'div.review-container')

for d in tqdm(reviews):
    try:
        sel2 = Selector(text = d.get_attribute('innerHTML'))
        try:
            rating = sel2.css('.rating-other-user-rating span::text').extract_first()
        except:
            rating = np.NaN
        try:
            review = sel2.css('.text.show-more__control::text').extract_first()
        except:
            review = np.NaN
        try:
            review_date = sel2.css('.review-date::text').extract_first()
        except:
            review_date = np.NaN
        try:
            author = sel2.css('.display-name-link a::text').extract_first()
        except:
            author = np.NaN
        try:
            review_title = sel2.css('a.title::text').extract_first()
        except:
            review_title = np.NaN
        try:
            review_url = sel2.css('a.title::attr(href)').extract_first()
        except:
            review_url = np.NaN
        rating_list.append(rating)
        review_date_list.append(review_date)
        review_title_list.append(review_title)
        author_list.append(author)
        review_list.append(review)
        review_url_list.append(review_url)
    except Exception as e:
        error_url_list.append(url)
        error_msg_list.append(e)
review_df = pd.DataFrame({
    'Review_Date':review_date_list,
    'Author':author_list,
    'Rating':rating_list,
    'Review_Title':review_title_list,
    'Review':review_list,
    'Review_Url':review_url
    })

review_df.to_csv('/content/reviews/barbie.csv')

100%|██████████| 175/175 [00:02<00:00, 77.69it/s] 


In [3]:
# Openheimer IMDB Reviews

url = "https://www.imdb.com/title/tt15398776/reviews"
driver.get(url)


sel = Selector(text = driver.page_source)
review_counts = sel.css('.lister .header span::text').extract_first().replace(',','').split(' ')[0]
more_review_pages = int(int(review_counts)/25)

for i in tqdm(range(more_review_pages)):
    try:
        css_selector = 'load-more-trigger'
        driver.find_element(By.ID, css_selector).click()
    except:
        pass

rating_list = []
review_date_list = []
review_title_list = []
author_list = []
review_list = []
review_url_list = []
error_url_list = []
error_msg_list = []
reviews = driver.find_elements(By.CSS_SELECTOR, 'div.review-container')

for d in tqdm(reviews):
    try:
        sel2 = Selector(text = d.get_attribute('innerHTML'))
        try:
            rating = sel2.css('.rating-other-user-rating span::text').extract_first()
        except:
            rating = np.NaN
        try:
            review = sel2.css('.text.show-more__control::text').extract_first()
        except:
            review = np.NaN
        try:
            review_date = sel2.css('.review-date::text').extract_first()
        except:
            review_date = np.NaN
        try:
            author = sel2.css('.display-name-link a::text').extract_first()
        except:
            author = np.NaN
        try:
            review_title = sel2.css('a.title::text').extract_first()
        except:
            review_title = np.NaN
        try:
            review_url = sel2.css('a.title::attr(href)').extract_first()
        except:
            review_url = np.NaN
        rating_list.append(rating)
        review_date_list.append(review_date)
        review_title_list.append(review_title)
        author_list.append(author)
        review_list.append(review)
        review_url_list.append(review_url)
    except Exception as e:
        error_url_list.append(url)
        error_msg_list.append(e)
review_df = pd.DataFrame({
    'Review_Date':review_date_list,
    'Author':author_list,
    'Rating':rating_list,
    'Review_Title':review_title_list,
    'Review':review_list,
    'Review_Url':review_url
    })


review_df.to_csv('/content/reviews/openheimer.csv')

100%|██████████| 150/150 [00:01<00:00, 102.59it/s]


## 1- Barbie Retriever

### Load data

In [4]:
!pip install wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11680 sha256=ce8749f9583adbc560d0407e8a8f9a272e5055b97b773839fe25bab982436854
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


In [5]:
from langchain.document_loaders import CSVLoader, WikipediaLoader

barbie_wiki_docs = WikipediaLoader(
    query = "Barbie (film)",
    load_max_docs = 1,
    doc_content_chars_max = 1_000_000
).load()

barbie_csv_docs = CSVLoader(
    file_path = "/content/reviews/barbie.csv",
    source_column = "Review_Url"
).load()

### Split data

In [18]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

wikipedia_text_splitter  = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 0,
    length_function = len,
    is_separator_regex = False,
    separators = ["\n==", "\n", " "]   # Keep headings, then paragraphs, then sentences
)

csv_text_splitter  = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50,
    length_function = len,
    is_separator_regex = False,
    separators = ["\n", " "]    # Keep paragraphes, then sentences
)

chunked_barbie_wiki_docs = wikipedia_text_splitter.transform_documents(
    barbie_wiki_docs
)

chunked_barbie_csv_docs = csv_text_splitter.transform_documents(
    barbie_csv_docs
)

### Embeddings and indexing

In [7]:
!pip install rank_bm25 tiktoken faiss-cpu

In [10]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
import os

os.environ["OPENAI_API_KEY"] = 'Enter your OPENAI_API_KEY'

embeddings = OpenAIEmbeddings()

barbie_csv_faiss_retriever = FAISS.from_documents(
    chunked_barbie_csv_docs,
    embeddings
).as_retriever() # barbie csv retriever

In [11]:
# BM25 Retriever
from langchain.retrievers import BM25Retriever, EnsembleRetriever

barbie_wikipedia_bm25_retriever = BM25Retriever.from_documents(
    chunked_barbie_wiki_docs
)
barbie_wikipedia_bm25_retriever.k = 1  # barbie wikipedia bm25 retriever

barbie_wikipedia_faiss_store = FAISS.from_documents(
    chunked_barbie_wiki_docs,
    embeddings
)   # barbie wiki faiss retriever

barbie_wikipedia_faiss_retriever = barbie_wikipedia_faiss_store.as_retriever(search_kwargs={"k": 1})

barbie_ensemble_retriever = EnsembleRetriever(
    retrievers=[barbie_wikipedia_bm25_retriever, barbie_wikipedia_faiss_retriever],
    weights=[0.25, 0.75] # should sum to 1
)

### Create retriever tool + Agent

In [13]:
# Tool

from langchain.agents.agent_toolkits import create_retriever_tool

barbie_wikipedia_retrieval_tool = create_retriever_tool(
    barbie_ensemble_retriever,
    "Wikipedia",
    "Searches and returns documents regarding the plot, history, and cast of the Barbie movie"
)

barbie_csv_retrieval_tool = create_retriever_tool(
    barbie_csv_faiss_retriever,
    "PublicReviews",
    "Searches and returns documents regarding public reviews of the Barbie movie"
)

barbie_retriever_tools = [barbie_wikipedia_retrieval_tool, barbie_csv_retrieval_tool]

In [14]:
# Agent

from langchain.agents.agent_toolkits import create_conversational_retrieval_agent
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(
    model = "gpt-3.5-turbo",
    temperature = 0
)

barbie_retriever_agent_executor = create_conversational_retrieval_agent(
    llm,
    barbie_retriever_tools,
    verbose = True
)


In [15]:
barbie_retriever_agent_executor({"input" : "Did people like Barbie, or did they find it too Philosphical? If they did, can you tell me why the movie is so Philosophical?"})



> Entering new AgentExecutor chain...

Invoking: `PublicReviews` with `{'query': 'Barbie movie reviews'}`


: 73
Review_Date: 29 July 2023
Author: planktonrules
Rating: 7
Review_Title: Good...not great
Review: "Barbie" is a movie that is setting all sorts of records, so it's obvious that the film is touching many people. For some, it's the fun of seeing a doll come to life, for others it's the theme of empowerment that make the movie a big hit. As for me, I did like it and thought it was a pretty good movie...but it's not for everyone.
Review_Url: /review/rw9209060/?ref_=tt_urv

: 120
Review_Date: 28 August 2023
Author: mateosilie
Rating: 5
Review_Title: Lack of Depth
Review: Let's start with what I enjoyed. The scenery, costumes and music were very nice but I believe that this movie was way over-hyped. I expected Barbie to be a touching movie that showed the positive and negative effects of this doll on society with a few FUNNY jokes here and there. Instead, I was greeted with an ov

{'input': 'Did people like Barbie, or did they find it too Philosphical? If they did, can you tell me why the movie is so Philosophical?',
 'chat_history': [HumanMessage(content='Did people like Barbie, or did they find it too Philosphical? If they did, can you tell me why the movie is so Philosophical?'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'PublicReviews', 'arguments': '{"query":"Barbie movie reviews"}'}}, response_metadata={'token_usage': <OpenAIObject at 0x7e1af6742cf0> JSON: {
    "prompt_tokens": 157,
    "completion_tokens": 17,
    "total_tokens": 174
  }, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3b956da36b', 'finish_reason': 'function_call', 'logprobs': None}, id='run-210bee9e-8213-4254-a314-7da25d50c772-0'),
  FunctionMessage(content=': 73\nReview_Date: 29 July 2023\nAuthor: planktonrules\nRating: 7\nReview_Title: Good...not great\nReview: "Barbie" is a movie that is setting all sorts of records, so it\'s obvious that the film 

In [16]:
barbie_retriever_agent_executor({"input" : "What is a very quick summary of the plot of the Barbie movie?"})



> Entering new AgentExecutor chain...

Invoking: `Wikipedia` with `{'query': 'Barbie movie plot'}`


Barbie is a 2023 fantasy comedy film directed by Greta Gerwig from a screenplay she wrote with Noah Baumbach. Based on the eponymous fashion dolls by Mattel, it is the first live-action Barbie film after numerous animated films and specials. It stars Margot Robbie as the title character and Ryan Gosling as Ken, and follows them on a journey of self-discovery through both Barbieland and the real world following an existential crisis. It is also a commentary regarding patriarchy and the effects of

comment) which comes closest to actually critiquing Barbie's legacy. ... And that's just it: the movie is hemmed in by the fact that it is, at the end of the day, produced by Mattel."The Barbie movie follows the title character, played by Margot Robbie, and Ken, played by Ryan Gosling, on a journey of self-discovery through both Barbieland and the real world following an existential crisis. T

{'input': 'What is a very quick summary of the plot of the Barbie movie?',
 'chat_history': [HumanMessage(content='Did people like Barbie, or did they find it too Philosphical? If they did, can you tell me why the movie is so Philosophical?'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'PublicReviews', 'arguments': '{"query":"Barbie movie reviews"}'}}, response_metadata={'token_usage': <OpenAIObject at 0x7e1af6742cf0> JSON: {
    "prompt_tokens": 157,
    "completion_tokens": 17,
    "total_tokens": 174
  }, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3b956da36b', 'finish_reason': 'function_call', 'logprobs': None}, id='run-210bee9e-8213-4254-a314-7da25d50c772-0'),
  FunctionMessage(content=': 73\nReview_Date: 29 July 2023\nAuthor: planktonrules\nRating: 7\nReview_Title: Good...not great\nReview: "Barbie" is a movie that is setting all sorts of records, so it\'s obvious that the film is touching many people. For some, it\'s the fun of seeing a dol

## 2- Openheimer Retriever

### Load data

In [17]:
oppenheimer_wikipedia_docs = WikipediaLoader(
    query = "Oppenheimer (film)",
    load_max_docs = 1,
    doc_content_chars_max = 1_000_000
).load()

oppenheimer_csv_docs = CSVLoader(
    file_path = "/content/reviews/openheimer.csv",
    source_column = "Review_Url"
).load()

### Split data

In [19]:
chunked_opp_wikipedia_docs = wikipedia_text_splitter.transform_documents(
    oppenheimer_wikipedia_docs
)

chunked_opp_csv_docs = csv_text_splitter.transform_documents(
    oppenheimer_csv_docs
)

### Embeddings + Indexing

In [20]:
opp_csv_faiss_retriever = FAISS.from_documents(
    chunked_opp_csv_docs,
    embeddings
).as_retriever()

# set up BM25 retriever
opp_wikipedia_bm25_retriever = BM25Retriever.from_documents(
    chunked_opp_wikipedia_docs
)
opp_wikipedia_bm25_retriever.k = 1

# set up FAISS vector store
opp_wikipedia_faiss_store = FAISS.from_documents(
    chunked_opp_wikipedia_docs,
    embeddings
)

opp_wikipedia_faiss_retriever = opp_wikipedia_faiss_store.as_retriever(search_kwargs={"k": 1})

# set up ensemble retriever
opp_ensemble_retriever = EnsembleRetriever(
    retrievers = [opp_wikipedia_bm25_retriever, opp_wikipedia_faiss_retriever],
    weights = [0.25, 0.75] # should sum to 1
)

### Multisource chain

In [21]:
from langchain.prompts import ChatPromptTemplate

system_message = """Use the information from the below two sources to answer any questions.

Source 1: public user reviews about the Oppenheimer movie
<source1>
{source1}
</source1>

Source 2: the wikipedia page for the Oppenheimer movie including the plot summary, cast, and production information
<source2>
{source2}
</source2>
"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_message),
         ("human", "{question}")
    ]
)

In [22]:
oppenheimer_multisource_chain = {
    "source1": (lambda x: x["question"]) | opp_ensemble_retriever,
    "source2": (lambda x: x["question"]) | opp_csv_faiss_retriever,
    "question": lambda x: x["question"],
} | prompt | llm

In [23]:
oppenheimer_multisource_chain.invoke({"question" : "What did people think of the Oppenheimer movie?"})

AIMessage(content="People generally had positive opinions about the Oppenheimer movie. Critics and audiences praised the film for its screenplay, cast performances, visuals, historical accuracy, score, cinematography, editing, and Christopher Nolan's direction. The movie received critical acclaim and was ranked as one of Nolan's best films and one of the best of 2023. It also received high ratings on review aggregator websites like Rotten Tomatoes. Additionally, the film won several awards, including Academy Awards, Golden Globe Awards, and British Academy Film Awards. Overall, the Oppenheimer movie was well-received by both critics and viewers.", response_metadata={'token_usage': <OpenAIObject at 0x7e1af83bd170> JSON: {
  "prompt_tokens": 2498,
  "completion_tokens": 122,
  "total_tokens": 2620
}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3b956da36b', 'finish_reason': 'stop', 'logprobs': None}, id='run-93f600dc-5ddf-4851-8c75-085cc18b35e7-0')

## Create Agent

In [26]:
from langchain.agents import Tool

def query_oppenheimer(input):
  return oppenheimer_multisource_chain.invoke(
      {
          "question": input
      }
  )

tools = [
    Tool(
        name = "BarbieInfo",
        func = barbie_retriever_agent_executor.invoke,
        description = "useful for when you need to answer questions about Barbie. Input should be a fully formed question."
    ),
    Tool(
        name = "OppenheimerInfo",
        func = query_oppenheimer,
        description = "useful for when you need to answer questions about Oppenheimer. Input should be a fully formed question."
    ),
]

In [27]:
from langchain.agents import ZeroShotAgent, AgentExecutor
from langchain import LLMChain

prefix = """Have a conversation with a human, answering the following questions as best you can. You have access to the following tools:"""
suffix = """Begin!"

Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix = prefix,
    suffix = suffix,
    input_variables=["input", "agent_scratchpad"]
)

llm_chain = LLMChain(
    llm = llm,
    prompt = prompt
)

In [28]:
barbenheimer_agent = ZeroShotAgent(
    llm_chain = llm_chain,
    tools = tools,
    verbose = True
)

barbenheimer_agent_chain = AgentExecutor.from_agent_and_tools(
    agent = barbenheimer_agent,
    tools = tools,
    verbose = True
)

In [29]:
barbenheimer_agent_chain.invoke({"input" : "What did people like about the Barbie movie?"})



> Entering new AgentExecutor chain...
Thought: I need to find out what people liked about the Barbie movie.
Action: BarbieInfo
Action Input: What did people like about the Barbie movie?

> Entering new AgentExecutor chain...
People liked the Barbie movie for various reasons. Some viewers enjoyed the fun of seeing a doll come to life, while others appreciated the theme of empowerment portrayed in the film. Additionally, some viewers found the depth and commentary on patriarchy appealing. The movie also received praise for its scenery, costumes, and music.

> Finished chain.

Observation: {'input': 'What did people like about the Barbie movie?', 'chat_history': [HumanMessage(content='Did people like Barbie, or did they find it too Philosphical? If they did, can you tell me why the movie is so Philosophical?'), AIMessage(content='', additional_kwargs={'function_call': {'name': 'PublicReviews', 'arguments': '{"query":"Barbie movie reviews"}'}}, response_metadata={'token_usage': <OpenAIOb

{'input': 'What did people like about the Barbie movie?',
 'output': 'People liked the Barbie movie for various reasons. Some viewers enjoyed the fun of seeing a doll come to life, while others appreciated the theme of empowerment portrayed in the film. Additionally, some viewers found the depth and commentary on patriarchy appealing. The movie also received praise for its scenery, costumes, and music.'}

In [30]:
barbenheimer_agent_chain.run({"input" : "What did people like about the Oppenheimer movie?"})



> Entering new AgentExecutor chain...
Thought: I need to find out what specific aspects of the Oppenheimer movie people enjoyed.
Action: OppenheimerInfo
Action Input: "What aspects of the Oppenheimer movie did people enjoy?"
Observation: content="People enjoyed various aspects of the Oppenheimer movie. Some of the aspects that received praise include:\n1. Screenplay\n2. Cast performances\n3. Visuals\n4. Historical accuracy\n5. Score\n6. Cinematography\n7. Editing\n8. Direction by Christopher Nolan\n\nAdditionally, specific performances by actors such as Cillian Murphy, Robert Downey Jr., and Emily Blunt were highlighted as exceptional. The film's ability to make a complex historical figure like J. Robert Oppenheimer's story compelling and engaging was also appreciated." response_metadata={'token_usage': <OpenAIObject at 0x7e1af50ba7f0> JSON: {
  "prompt_tokens": 2472,
  "completion_tokens": 113,
  "total_tokens": 2585
}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3b956d

"People enjoyed various aspects of the Oppenheimer movie, including the screenplay, cast performances, visuals, historical accuracy, score, cinematography, editing, and direction by Christopher Nolan. Specific performances by actors like Cillian Murphy, Robert Downey Jr., and Emily Blunt were also highlighted as exceptional. The film's ability to make J. Robert Oppenheimer's story compelling and engaging was appreciated."

In [31]:
barbenheimer_agent_chain.run({"input" : "Did the movies Barbie and Oppenheimer share similar themes or ideas?"})



> Entering new AgentExecutor chain...
Thought: I need to gather information about both Barbie and Oppenheimer to see if they share similar themes or ideas.
Action: BarbieInfo
Action Input: "What are some common themes in Barbie movies?"

> Entering new AgentExecutor chain...

Invoking: `Wikipedia` with `{'query': 'Common themes in Barbie movies'}`


Barbie.

it's like being a woman today while providing more than a laugh or two in the process." Chinese film critic Li Jingfei (李竞菲) praised the film's humor and criticism of patriarchy, qualifying that its themes were too often advanced by slogans rather than the experiences of its characters.Some common themes in Barbie movies include:

1. Empowerment: Many Barbie movies portray themes of empowerment, encouraging viewers to believe in themselves and their abilities.

2. Friendship: Friendship is a recurring theme in Barbie movies, emphasizing the importance of loyalty, support, and camaraderie.

3. Adventure: Barbie movies often featur

"Based on the information gathered, Barbie movies focus on themes like empowerment, friendship, adventure, self-discovery, and overcoming challenges, while Oppenheimer's work explores ethical dilemmas, internal conflict, historical context, human nature, and the complexity of mankind. While there may be some overlap in themes like personal struggles and ethical dilemmas, the overall themes and ideas in Barbie and Oppenheimer's work are quite different."